In [1]:
import requests
from bs4 import BeautifulSoup

import re
import pandas as pd

Fill in scraping information here

In [50]:
season = "fuyu"
season_jap = "冬"
season_url = "https://ouchidehaiku.com/winter/contents/seasonwords"
number_of_pages = 368

In [51]:
# Function to scrape kigo page (excluding pronunciation)

def scrape_page_kigo(url)  -> list:
    response = requests.get(url)   # this gets the URL
    source = response.text        # .text is an attribute, it gets the text
    soup = BeautifulSoup(source, "lxml")
    
    text= soup.find('ol').find_all("li")

    kigo_list=[]
    for element in text:
        kigo_and_pronunciation = element.find("h3").get_text()
        kigo = re.sub(r"""（.*）""", "", kigo_and_pronunciation)   # you need to remove the full-width Japanese parenthesis

        kigo_list.append(kigo)

    return kigo_list

In [52]:
# Function to scrape kigo page (including pronunciation)

def scrape_page_pronunciation(url)  -> list:
    response = requests.get(url)   # this gets the URL
    source = response.text        # .text is an attribute, it gets the text
    soup = BeautifulSoup(source, "lxml")
    
    text= soup.find('ol').find_all("li")

    pronunciation_list = []
    for element in text:
        kigo_and_pronunciation = element.find("h3").get_text()

        pronunciation = re.sub(r"""）""", "", kigo_and_pronunciation)
        pronunciation = re.sub(r""".*（""", "", pronunciation)
        pronunciation_list.append(pronunciation)

    return pronunciation_list  

In [53]:
scrape_page_kigo(season_url)

['愛日', 'アイヌ山葵', '相嘗祭', '青木の実', '青頸', '青首大根', '石蓴', '青鮫', '青写真', '青摺']

In [54]:
scrape_page_pronunciation(season_url)

['あいじつ',
 'あいぬわさび',
 'あいんべのまつり',
 'あおきのみ',
 'あおくび',
 'あおくびだいこん',
 'あおさ',
 'あおざめ',
 'あおじゃしん',
 'あおずり']

In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# driver = webdriver.Chrome()  # no need for executable path any more

Spring (haru): 427 pages

Summer (natsu): 591 pages

Autumn (aki): 474 pages

Winter (fuyu): 368 pages

New Year (shinnen): 229 pages

In [23]:
driver = webdriver.Chrome() 
driver.get("https://ouchidehaiku.com/newyear/contents/seasonwords/page/3")

link = driver.find_element(By.CSS_SELECTOR, "span.next")  # click to get to next page
link.click()

In [20]:
# FUNCTION 1

# click on next page button and scrape page, return list of kigo and list of pronunciations

def click_and_scrape_page(url, num_pages) -> list:

    kigo_list = scrape_page_kigo(url)                       # starting list for kigo
    pronunciation_list = scrape_page_pronunciation(url)     # starting list for pronunciation

    driver = webdriver.Chrome() 
    driver.get(url)

    for i in range(num_pages):
        if i > 0:
            link = driver.find_element(By.CLASS_NAME, "next")  # click to get to next page
            link.click()
            current_url = driver.current_url

            response = requests.get(current_url)   # this gets the URL
            source = response.text        # .text is an attribute, it gets the text
            soup = BeautifulSoup(source, "lxml")
            
            text= soup.find('ol').find_all("li")

            for element in text:
                kigo_and_pronunciation = element.find("h3").get_text()
                kigo = re.sub(r"""（.*）""", "", kigo_and_pronunciation)   # you need to remove the full-width Japanese parenthesis

                kigo_list.append(kigo)

                pronunciation = re.sub(r"""）""", "", kigo_and_pronunciation)
                pronunciation = re.sub(r""".*（""", "", pronunciation)
                pronunciation_list.append(pronunciation)  
            
                i = i-1

    return kigo_list, pronunciation_list
    

In [56]:
# FUNCTION 2 (for Winter and New_Year)

# click on next page button and scrape page, return list of kigo and list of pronunciations

def click_and_scrape_page2(url, num_pages) -> list:

    kigo_list = scrape_page_kigo(url)                       # starting list for kigo
    pronunciation_list = scrape_page_pronunciation(url)     # starting list for pronunciation

    driver = webdriver.Chrome() 
    driver.get(url)

    for i in range(num_pages):
        if i > 0:
            link = driver.find_element(By.CSS_SELECTOR, "span.next")  # click to get to next page
            link.click()
            current_url = driver.current_url

            response = requests.get(current_url)   # this gets the URL
            source = response.text        # .text is an attribute, it gets the text
            soup = BeautifulSoup(source, "lxml")
            
            text= soup.find('ol').find_all("li")

            for element in text:
                kigo_and_pronunciation = element.find("h3").get_text()
                kigo = re.sub(r"""（.*）""", "", kigo_and_pronunciation)   # you need to remove the full-width Japanese parenthesis

                kigo_list.append(kigo)

                pronunciation = re.sub(r"""）""", "", kigo_and_pronunciation)
                pronunciation = re.sub(r""".*（""", "", pronunciation)
                pronunciation_list.append(pronunciation)  
            
                i = i-1

    return kigo_list, pronunciation_list

In [57]:
fuyu_scraping = click_and_scrape_page2(season_url, number_of_pages) 

In [58]:
# write to csv file

df = pd.DataFrame({"kigo": fuyu_scraping[0],
                   "pronunciation": fuyu_scraping[1],
                  "season": season_jap})

df.to_csv("./" + season + "_kigo_dataset.csv", index=False)